In [1]:
import numpy as np
import pandas as pd
import datetime
import os
from sklearn.model_selection import train_test_split
# from tqdm import tqdm

In [3]:
BASE_PATH = 'C:/Users/sjana/Learning/LJMU-masters/recommender_system/workspace/rest_procssed_data/'
rest_emd_path = BASE_PATH+'rest_context_pca_feat.csv'
dqn_train_data_path = BASE_PATH+'dqn_offline_train_validation.csv'
simulation_data_path = BASE_PATH+'dqn_simulation_data.csv'

SEED = 133610

In [21]:
rest_embedding_cdf = pd.read_csv(rest_emd_path)
print(rest_embedding_cdf.shape)
rest_embedding_cdf = rest_embedding_cdf.set_index('business_id')
rest_embedding_cdf.head()

(19590, 7)


,0,1,2,3,4,5
business_id,,,,,,
QXAEGFB4oINsVuTFxEYKFQ,15.226513,1.540680,-2.398470,1.394465,0.389276,0.359321
1Dfx3zM-rW4n-31KeC8sJg,-18.362388,-2.902234,0.897339,-1.279854,-1.003267,0.525670
1RHY4K3BD22FK7Cfftn8Mg,14.163315,-1.337360,2.580457,0.243556,0.209694,0.093695
tstimHoMcYbkSC4eBA1wEg,-20.902280,0.200356,-1.117485,0.266005,0.375828,-0.212620
NDuUMJfrWk52RA-H-OtrpA,15.512376,1.656747,2.486269,0.953304,-0.016490,-0.089844


In [22]:
list(rest_embedding_cdf.loc['QXAEGFB4oINsVuTFxEYKFQ'])

[15.226512675755195,
 1.5406801706788598,
 -2.398469549279876,
 1.394465007714395,
 0.38927558224578096,
 0.3593209300076965]

In [5]:
dqn_data_df = pd.read_csv(simulation_data_path)
dqn_data_df['state'] = dqn_data_df.state. \
    apply(lambda x: np.asarray([each.split('|') for each in x.split(',')], dtype='float32'))
dqn_data_df['action'] = dqn_data_df.action.apply(lambda x: np.asarray(x.split('|'), dtype='float32'))
dqn_data_df['reward'] = dqn_data_df.reward.astype(float)

In [6]:
dqn_data_df.head()

,user_id,state,action,reward,next_state
0,---1lKK3aKOuomHnwAkAow,"[[5.105682, 3.02343, 1.1998029, 2.6459227, 5.9...","[3.907493, 3.9790266, 0.85891354, 2.9955611, 6...",4.0,3.6259165|4.555624|1.7463031000000002|3.389558...
1,---1lKK3aKOuomHnwAkAow,"[[3.6259165, 4.555624, 1.7463031, 3.3895586, 7...","[2.1000068, 5.064063, 1.3642833, 3.633889, 7.8...",4.0,5.943884400000001|2.053497|2.2759242000000004|...
2,---1lKK3aKOuomHnwAkAow,"[[5.9438844, 2.053497, 2.2759242, 2.484219, 7....","[2.4830651, 5.704142, 0.17315382, 4.1524696, 6...",5.0,3.21387|4.1962743|2.4558017000000003|3.489384|...
3,---1lKK3aKOuomHnwAkAow,"[[3.21387, 4.1962743, 2.4558017, 3.489384, 8.5...","[1.5584882, 4.7903037, 0.621162, 4.0098743, 6....",5.0,5.105682|3.02343|1.1998029|2.6459227000000003|...
4,---1lKK3aKOuomHnwAkAow,"[[5.105682, 3.02343, 1.1998029, 2.6459227, 5.9...","[4.054323, 6.1892066, 0.6355051, 3.7389784, 7....",2.0,5.105682|3.02343|1.1998029|2.6459227000000003|...


### Split the data into training and testing

In [11]:
X = dqn_data_df[['state','action']]
Y = dqn_data_df.reward

In [15]:
TEST_SIZE = 0.15
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=TEST_SIZE, random_state=SEED)

In [16]:
VALID_SIZE = 0.15
X_train, X_validation, Y_train, Y_validation = train_test_split(X_train, Y_train, test_size=VALID_SIZE, random_state=SEED)

In [17]:
# print number of samples in each set
print("TRAINING DATA")
print('Shape of input sequences: {}'.format(X_train.shape))
print('Shape of output sequences: {}'.format(Y_train.shape))
print("-"*50)
print("VALIDATION DATA")
print('Shape of input sequences: {}'.format(X_validation.shape))
print('Shape of output sequences: {}'.format(Y_validation.shape))
print("-"*50)
print("TESTING DATA")
print('Shape of input sequences: {}'.format(X_test.shape))
print('Shape of output sequences: {}'.format(Y_test.shape))

TRAINING DATA
Shape of input sequences: (175335, 2)
Shape of output sequences: (175335,)
--------------------------------------------------
VALIDATION DATA
Shape of input sequences: (30942, 2)
Shape of output sequences: (30942,)
--------------------------------------------------
TESTING DATA
Shape of input sequences: (36402, 2)
Shape of output sequences: (36402,)


In [18]:
STATE_WINDOW=5
FEATURE_SPACE=6

BATCH_SIZE=128

## Create Generator function for traing and validation data

In [37]:
def generateStateActionMatrix(x_data, y_data, batch_size):
    while True:
        data_size = x_data.shape[0]
        
        for index in range(0, data_size, batch_size):
            #create the x_return with shape (batch_size, )
            x_temp = x_data[index:index+batch_size]
            x_state_input = np.zeros((x_temp.shape[0],STATE_WINDOW,FEATURE_SPACE))
            x_action_input = np.zeros((x_temp.shape[0],FEATURE_SPACE))
#             for i,each_row in enumerate(x_temp.iterrows()):
#                 state_array = list(each_row[1]['state'].split(',')) 
#                 state_data = np.array([rest_embedding_cdf.loc[bid] for bid in state_array])
#                 x_state_input[i,:,:] = state_data
                
#                 action_data = np.array(rest_embedding_cdf.loc[str(each_row[1]['action'])]) 
#                 x_action_input[i,:] = action_data
            
            y_temp = y_data[index:index+batch_size]
            y_input = np.array([float(bid) for bid in list(y_temp)])
#             print('batch size {}, x_state_input shape {}, x_action_input shape {} and y_input shape {} '\
#                   .format(batch_size, x_state_input.shape, x_action_input.shape, y_input.shape))
            yield [x_state_input, x_action_input], y_input       

In [38]:
unit_test_x = X_train[:12]
unit_test_y = Y_train[:12]

for _ in range(3):
    gen_func = generateStateActionMatrix(unit_test_x,unit_test_y,5)
    x_input, y_rating = next(gen_func)
    print('shapes of state {} - action {} - reward {}'.format(x_input[0].shape, x_input[1].shape, y_rating.shape))

shapes of state (5, 5, 6) - action (5, 6) - reward (5,)
shapes of state (5, 5, 6) - action (5, 6) - reward (5,)
shapes of state (5, 5, 6) - action (5, 6) - reward (5,)


## Build the DQN model Architecture

In [39]:
import warnings
warnings.filterwarnings('ignore')
import keras
print(keras.__version__)

2.2.4


In [40]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, GRU, SimpleRNN, Concatenate, Dense, Flatten,BatchNormalization, Dropout, TimeDistributed, Lambda
from keras.regularizers import l2
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint
import keras.backend as K

In [51]:
# Critic model for the ddpg
def create_dqn_model_arch1():
    state_input = Input(shape=(STATE_WINDOW,FEATURE_SPACE,),name='state_input')
    state_gru = GRU(32,return_sequences=True, kernel_initializer='random_uniform', name='state_gru')(state_input)
    state_dense = Dense(32, activation='relu', kernel_initializer='random_uniform', name='state_dense')(state_gru)
    state_flatten = Flatten(name='state_flatten')(state_dense)
    
    action_input = Input(shape=(FEATURE_SPACE,),name='action_input')
    action_dense = Dense(16, activation='relu', kernel_initializer='random_uniform', name='action_dense')(action_input)
    
    dense_conc = Concatenate()([state_flatten, action_dense])
    hidden_layer = Dense(32, activation='relu',kernel_initializer='random_uniform')(dense_conc)
#     dp = BatchNormalization()(hidden_layer)
#     drop_out = Dropout(0.25)(hidden_layer)
    out_layer = Dense(1, activation='linear', kernel_initializer='random_uniform')(hidden_layer)
    
    model = Model(inputs=[state_input, action_input], outputs=out_layer)
    model.compile(
      loss='mse',
      optimizer=Adam(lr=0.02),
#       optimizer=SGD(lr=0.01, momentum=0.9),
      metrics=['mse'],
    )
    return model

action_grads_critic = K.function([dqn_model.input[0], dqn_model.input[1]], 
                                 K.gradients(dqn_model.output, [dqn_model.input[1]]))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
state_input (InputLayer)        (None, 5, 6)         0                                            
__________________________________________________________________________________________________
state_gru (GRU)                 (None, 5, 32)        3744        state_input[0][0]                
__________________________________________________________________________________________________
state_dense (Dense)             (None, 5, 32)        1056        state_gru[0][0]                  
__________________________________________________________________________________________________
action_input (InputLayer)       (None, 6)            0                                            
__________________________________________________________________________________________________
state_flat

In [65]:
def create_dqn_model_arch2():
    state_input = Input(shape=(STATE_WINDOW,FEATURE_SPACE,),name='state_input')
#     state_gru = SimpleRNN(32,return_sequences=False, kernel_initializer='random_uniform', name='state_gru')(state_input)
    state_dense = Flatten(name='state_flatten')(state_input)
    
    action_input = Input(shape=(FEATURE_SPACE,),name='action_input')
#     action_dense = Dense(32, activation='relu', kernel_initializer='random_uniform', name='action_dense')(action_input)
    
    dense_conc = Concatenate()([state_dense,action_input])
    hidden_layer1 = Dense(64, activation='relu')(dense_conc)
    drop_out_1 = Dropout(0.25)(hidden_layer1)
    hidden_layer2 = Dense(32, activation='relu')(drop_out_1)
    drop_out_2 = Dropout(0.25)(hidden_layer2)
    out_layer = Dense(1, activation='linear')(drop_out_2)
    
    model = Model(inputs=[state_input, action_input], outputs=out_layer)
    model.compile(
      loss='mse',
#       optimizer=Adam(lr=0.01),
      optimizer=SGD(lr=0.01, momentum=0.9),
      metrics=['mse'],
    )
    return model

In [60]:
def actor_optimizer():
    action_gdts = K.placeholder(shape=(None, FEATURE_SPACE))
    params_grad = tf.gradients(actor.output, actor.trainable_weights, -action_gdts)
    grads = zip(params_grad, actor.trainable_weights)
#     print(list(grads))
    return K.function([actor.input, action_gdts], [tf.train.AdamOptimizer(0.01).apply_gradients(grads)])

def create_actor():
    state_input = Input(shape=(STATE_WINDOW,FEATURE_SPACE,),name='state_input')
    state_gru = GRU(32,return_sequences=True, kernel_initializer='random_uniform', name='state_gru')(state_input)
    state_dense = Dense(32, activation='relu', kernel_initializer='random_uniform', name='state_dense')(state_gru)
    state_flatten = Flatten(name='state_flatten')(state_dense)
    state_dense2 = Dense(FEATURE_SPACE, activation='tanh', kernel_initializer='random_uniform', name='state_dense2')(state_flatten)
    out = Lambda(lambda i: i * FEATURE_SPACE)(state_dense2)
    model = Model(inputs=state_input, outputs=out)
    return model
    
actor = create_actor()
actor_optimizer = actor_optimizer()
actor.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
state_input (InputLayer)     (None, 5, 6)              0         
_________________________________________________________________
state_gru (GRU)              (None, 5, 32)             3744      
_________________________________________________________________
state_dense (Dense)          (None, 5, 32)             1056      
_________________________________________________________________
state_flatten (Flatten)      (None, 160)               0         
_________________________________________________________________
state_dense2 (Dense)         (None, 6)                 966       
_________________________________________________________________
lambda_7 (Lambda)            (None, 6)                 0         
Total params: 5,766
Trainable params: 5,766
Non-trainable params: 0
_________________________________________________________________


In [66]:
# dqn_model = create_dqn_model_arch1()
dqn_model = create_dqn_model_arch2()
print(dqn_model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
state_input (InputLayer)        (None, 5, 6)         0                                            
__________________________________________________________________________________________________
state_flatten (Flatten)         (None, 30)           0           state_input[0][0]                
__________________________________________________________________________________________________
action_input (InputLayer)       (None, 6)            0                                            
__________________________________________________________________________________________________
concatenate_8 (Concatenate)     (None, 36)           0           state_flatten[0][0]              
                                                                 action_input[0][0]               
__________

## Fit the model

In [67]:
# X_train, X_validation, Y_train, Y_validation
train_generator = generateStateActionMatrix(X_train, Y_train, BATCH_SIZE)
val_generator = generateStateActionMatrix(X_validation, Y_validation, BATCH_SIZE)

In [23]:
# for j, each in enumerate(range(steps_per_epoch)):
#     d1, d2 = next(train_generator)
#     print('{} - state {} - action {} - reward {}'.format(j, d1[0].shape, d1[1].shape, d2.shape))

In [68]:
curr_dt_time = datetime.datetime.now()
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{mean_squared_error:.5f}-{val_mean_squared_error:.5f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=5, min_lr=0.001)# write the REducelronplateau code here
callbacks_list = [checkpoint]

In [69]:
num_train_sequences = X_train.shape[0]
print('# training sequences =', num_train_sequences)
num_val_sequences = X_validation.shape[0]
print('# validation sequences =', num_val_sequences)
num_epochs = 15 # choose the number of epochs
print ('# epochs =', num_epochs)

if (num_train_sequences%BATCH_SIZE) == 0:
    steps_per_epoch = int(num_train_sequences/BATCH_SIZE)
else:
    steps_per_epoch = (num_train_sequences//BATCH_SIZE) + 1

if (num_val_sequences%BATCH_SIZE) == 0:
    validation_steps = int(num_val_sequences/BATCH_SIZE)
else:
    validation_steps = (num_val_sequences//BATCH_SIZE) + 1
    
print(steps_per_epoch)
print(validation_steps)

# training sequences = 175335
# validation sequences = 30942
# epochs = 15
1370
242


In [70]:
dqn_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/15
1370/1370 [==============================] - 7s 5ms/step - loss: 1.4192 - mean_squared_error: 1.4192 - val_loss: 1.3667 - val_mean_squared_error: 1.3667

Epoch 00001: saving model to model_init_2020-01-3120_38_47.063610/model-00001-1.41925-1.36672.h5
Epoch 2/15
1370/1370 [==============================] - 5s 4ms/step - loss: 1.3455 - mean_squared_error: 1.3455 - val_loss: 1.3667 - val_mean_squared_error: 1.3667

Epoch 00002: saving model to model_init_2020-01-3120_38_47.063610/model-00002-1.34549-1.36672.h5
Epoch 3/15
1370/1370 [==============================] - 5s 4ms/step - loss: 1.3455 - mean_squared_error: 1.3455 - val_loss: 1.3667 - val_mean_squared_error: 1.3667

Epoch 00003: saving model to model_init_2020-01-3120_38_47.063610/model-00003-1.34549-1.36672.h5
Epoch 4/15
1370/1370 [==============================] - 7s 5ms/step - loss: 1.3455 - mean_squared_error: 1.3455 - val_loss: 1.3667 - val_mean_squared_error: 1.3667

Epoch 00004: saving model to model_init_2020-01-3

## Perform Testing using RL

In [71]:
test_generator = generateStateActionMatrix(X_test, Y_test, BATCH_SIZE)
num_test_sequences = X_test.shape[0]
print('# testing sequences =', num_test_sequences)

if (num_test_sequences%BATCH_SIZE) == 0:
    testing_steps = int(num_test_sequences/BATCH_SIZE)
else:
    testing_steps = (num_test_sequences//BATCH_SIZE) + 1

# testing sequences = 36402


In [72]:
test_loss = list()
test_acc = list()
for n in range(num_test_sequences):
    input_data, y_true = next(test_generator)
    score = dqn_model.evaluate(input_data, y_true, verbose=0)
#     print('test loss {} and test accurancy {}'.format(score[0], score[1]))
    test_loss.append(score[0])
    test_acc.append(score[1])
    
print('average loss --', np.mean(test_loss))
# print('average accuracy --', np.mean(test_acc))

average loss -- 1.3536713173297938
